Для початку завантажую всі необхідні бібліотеки

In [44]:
from tensorflow.keras.models import load_model
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image

Далі завантажую дані для тесту моделі

In [18]:
(_, _), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 4s 0us/step


Приводжу дані до необхідної форми

In [19]:
X_test = X_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
y_test = to_categorical(y_test)

Далі завантажую модель

In [8]:
cnn = load_model('mnist_cnn.h5')

Тестую модель на завантажених даних

In [20]:
loss, accuracy = cnn.evaluate(X_test, y_test)

print(f'Test accuracy: {accuracy * 100:.2f}%')


313/313 [==============================] - 5s 12ms/step - loss: 0.0342 - accuracy: 0.9894
Test accuracy: 98.94%


Ще раз для тесту завантажую зображення цифри і приводжу його до необхідної форми

In [45]:
img_path = '7.png'
img = Image.open(img_path)
img = img.convert('L')
img = img.resize((28, 28))
img_array = image.img_to_array(img)
img_array = img_array.reshape(1, 28, 28, 1).astype('float32') / 255

Тестую модель на завантаженому зображенні

In [46]:
predictions = cnn.predict(img_array)

for index, probability in enumerate(predictions[0]):
    print(f'{index}: {probability:.10%}')


predicted_class = np.argmax(predictions[0])
print(f'The predicted class is: {predicted_class}')

1/1 [==============================] - 0s 28ms/step
0: 0.0009095054%
1: 0.0016049002%
2: 0.1096770633%
3: 0.0007550132%
4: 0.0033071283%
5: 0.0003801694%
6: 0.0000564639%
7: 99.8699784279%
8: 0.0039431350%
9: 0.0093937779%
The predicted class is: 7


Далі за допомогою бібліотеки gradio створюю інтерфейс для роботи з моделлю, в ньому можна завантажити фото і розпізнати його за допомогою моделі

In [48]:
def preprocess_image(image):
    img_pil = Image.fromarray(image)
    img_resized = img_pil.resize((28, 28))
    img_array = np.array(img_resized).mean(axis=-1).reshape(1, 28, 28, 1) / 255.0
    return img_array

def predict_digit(image):
    img_array = preprocess_image(image)
    prediction = cnn.predict(img_array)
    predicted_label = np.argmax(prediction[0])
    return f'The predicted number is: {predicted_label}'

iface = gr.Interface(
    fn=predict_digit,
    inputs="image",
    outputs="label"
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7871
Running on public URL: https://a9bf1a229d9f671f48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
